In [12]:
import cv2
import tensorflow as tf
import numpy as np
model = tf.keras.models.load_model("CNNDISTANCE2.model")

In [13]:
def prepare(ima):
    IMG_SIZE = 100# 50 in txt-based
    img_array=ima*255
    #img_array = cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    img_array=img_array/255.0  # filepathread in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1,IMG_SIZE, IMG_SIZE,1)

In [14]:
def paa_skin(image,image1):  
    rw=300
    rh=300
    cx=100
    cy=100
    hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    ycbcr=cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    b,g,r=cv2.split(image)
    y,cr,cb=cv2.split(ycbcr)
    h,s,v=cv2.split(hsv)
    s=s/(h+s+v)
    paa=((0<=h)*(h<=50)*(0.23<s)*(s<0.68)*(r>95)*(g>40)*(b>20)*(r>g)*(r>b)*(abs(r-g)>15))+((r>95)*(g>40)*(b>20)*(r>g)*(r>b)*(abs(r-g)>15)*(cr>135)*(cb>85)*(y>80)*(cr<=(1.5862*cb+20))*(cr>=(0.3448*cb+76.2069))*(cr>=(-4.5652*cb+234.5652))*(cr<=-1.15*cb+301.75)*(cr<=-2.2857*cb+432.85))
    b=b*paa
    g=g*paa
    r=r*paa
    new=cv2.merge((b,g,r))
    paa=paa*255
    new1=cv2.merge((paa,paa,paa))
    cv2.imwrite("/home/paa/55.jpeg",new1)
    new1=cv2.imread("/home/paa/55.jpeg")
    new1 = cv2.cvtColor(new1, cv2.COLOR_BGR2GRAY)
    dist = cv2.distanceTransform(new1, cv2.DIST_L2, 5)
    cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
    return image1,dist


In [15]:
cap=cv2.VideoCapture(0)
cx=100
cy=100
rw=300
rh=300
while True:
    ret, image=cap.read()
    pre=image[cx:rw,cy:rh]
    image1,dist=paa_skin(pre,image)
    prediction = model.predict([prepare(dist)])
    prediction=np.argmax(prediction)
    x1=prediction
    try:
        cv2.putText(image,str(x1),(60,60),cv2.FONT_HERSHEY_SIMPLEX,3.0,(255,0,0),lineType=cv2.LINE_AA)
    except:
        print("Variable x1 is not empty")
    cv2.rectangle(image,(cx,cy),(cx+rw,cy+rh),(0,0,0),5)
    cv2.imshow('dist',dist)
    cv2.imshow('image1',image1)
    keypress = cv2.waitKey(1) & 0xFF
    if keypress == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

/home/paa/anaconda3/envs/opencv-env/lib/python2.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in divide
  # This is added back by InteractiveShellApp.init_path()
